In [39]:
# -*- coding: utf-8
# Transparency International - Brazil (https://transparenciainternacional.org.br)
# Reinaldo Chaves (@paidatocandeira)
# Trata as bases de informações do SurveyMonkey (registro de candidatos que aderiram à campanha) e do Pacto pela Democracia (registro de candidatos que aderiram)
#

In [ ]:
import pandas as pd
from datetime import date
import math

# Importa Survey

In [40]:
survey = pd.read_csv("monkey/CADASTRO DE CANDIDATASOS UNIDOS CONTRA A CORRUP_5_out2.csv",sep=',',encoding = 'utf-8',  converters={"CPF (apenas números)": lambda x: str(x),
                                                                                                                                "Número do Título de Eleitor\xa0(apenas números)": lambda x: str(x)})

In [ ]:
# Lista nomes originais de colunas

In [41]:
columnNames = list(survey.head(0)) 
print(columnNames)

['respondent_id', 'collector_id', 'date_created', 'date_modified', 'ip_address', 'email_address', 'first_name', 'last_name', 'custom_1', 'Nome completo (como está registrado na justiça eleitoral)', 'E-mail\xa0de contato', 'Cargo pretendido', 'CPF (apenas números)', 'Número do Título de Eleitor\xa0(apenas números)', 'Nome que\xa0aparecerá na urna', 'Número que\xa0aparecerá na urna', 'UF', 'Município em que nasceu', 'Ano de nascimento', 'Ocupação profissional', 'Partido político', 'Endereços de suas redes sociais e site (se tiver)', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Endereço do site de sua campanha de financiamento coletivo (se tiver)', 'Declaro que as 24 afirmações acima são verdadeiras sem ressalvas e por isso me enquadro no critério de "passado limpo"', 'Por fim, declaro que: (escolha apenas uma opção)Esta informação não desqualifica o "passado limpo" do/a candidato/a, mas ficará disponível para o/a eleitor/a', 'Caso tenha respondido na pergunta anterior que

In [42]:
survey.rename(columns={"Nome completo (como está registrado na justiça eleitoral)": "nome_completo"}, inplace=True)
survey.rename(columns={"E-mail\xa0de contato": "e_mail_survey"}, inplace=True)
survey.rename(columns={"CPF (apenas números)": "cpf"}, inplace=True)
survey.rename(columns={"Ocupação profissional": "ocupacao_survey"}, inplace=True)
survey.rename(columns={"Número do Título de Eleitor\xa0(apenas números)": "titulo_eleitor"}, inplace=True)
survey.rename(columns={"Nome que\xa0aparecerá na urna": "nome_na_urna"}, inplace=True)
survey.rename(columns={"Número que\xa0aparecerá na urna": "numero_na_urna"}, inplace=True)
survey.rename(columns={"Município em que nasceu": "municipio_nascimento"}, inplace=True)
survey.rename(columns={"Endereços de suas redes sociais e site (se tiver)": "site_candidato"}, inplace=True)
survey.rename(columns={"Unnamed: 22": "url_facebook"}, inplace=True)
survey.rename(columns={"Unnamed: 23": "url_twitter"}, inplace=True)
survey.rename(columns={"Unnamed: 24": "url_instagram"}, inplace=True)
survey.rename(columns={"Unnamed: 25": "url_youtube"}, inplace=True)
survey.rename(columns={"Endereço do site de sua campanha de financiamento coletivo (se tiver)": "financiamento_coletivo"}, inplace=True)
survey.rename(columns={'Declaro que as 24 afirmações acima são verdadeiras sem ressalvas e por isso me enquadro no critério de "passado limpo"': 'passado_limpo'}, inplace=True)
survey.rename(columns={'Por fim, declaro que: (escolha apenas uma opção)Esta informação não desqualifica o "passado limpo" do/a candidato/a, mas ficará disponível para o/a eleitor/a': "processos_primeira_instancia"}, inplace=True)
survey.rename(columns={"Caso tenha respondido na pergunta anterior que já sofreu alguma sanção, você pode incluir comentários no campo abaixo (máximo de 500 caracteres):": "detalhes_processos_survey"}, inplace=True)
survey.rename(columns={"Eu já assinei o Pacto pela Democracia": "pacto"}, inplace=True)
survey.rename(columns={"Eu apoio as Novas Medidas Contra a Corrupção e, caso eleita/o, trabalharei já no início do meu mandato para colocar as propostas em tramitação e atuarei para a sua aprovação": "aderiu_totalmente_medidas"}, inplace=True)
survey.rename(columns={"Manifesto ressalva sobre as seguintes medidas e justifico (máximo de 500 caracteres):": "justificativa_adesao_medidas"}, inplace=True)

In [ ]:
# Deixa coluna em formato datetime

In [43]:
survey['date_created'] = pd.to_datetime(survey['date_created'])

In [44]:
survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 33 columns):
respondent_id                   2060 non-null float64
collector_id                    2060 non-null float64
date_created                    2060 non-null datetime64[ns]
date_modified                   2060 non-null object
ip_address                      2060 non-null object
email_address                   0 non-null float64
first_name                      0 non-null float64
last_name                       0 non-null float64
custom_1                        0 non-null float64
nome_completo                   2061 non-null object
e_mail_survey                   2061 non-null object
Cargo pretendido                2061 non-null object
cpf                             2061 non-null object
titulo_eleitor                  2061 non-null object
nome_na_urna                    956 non-null object
numero_na_urna                  956 non-null object
UF                              956 non-null 

In [45]:
survey.reset_index().head()

,index,respondent_id,collector_id,date_created,date_modified,ip_address,email_address,first_name,last_name,custom_1,...,url_twitter,url_instagram,url_youtube,financiamento_coletivo,passado_limpo,processos_primeira_instancia,detalhes_processos_survey,pacto,aderiu_totalmente_medidas,justificativa_adesao_medidas
0,0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,Twitter,Instagram,Youtube,Open-Ended Response,Response,Response,Open-Ended Response,Response,Response,Open-Ended Response
1,1,1.026128e+10,215760080.0,2018-10-05 20:58:06,10/05/2018 08:59:11 PM,191.255.242.55,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1.026124e+10,215760080.0,2018-10-05 20:35:50,10/05/2018 08:37:46 PM,200.174.132.2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1.026120e+10,215760080.0,2018-10-05 20:14:09,10/05/2018 08:14:58 PM,179.176.175.61,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1.026119e+10,215760080.0,2018-10-05 20:11:03,10/05/2018 08:13:02 PM,179.159.23.23,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
# Preenche zeros a esquerda de cpf
def compara1(linha):
    documento = str(linha['cpf']).strip()
    if documento == 'Open-Ended Response' or documento == "":
        alterado = documento
    if len(documento) < 11:
        conta = 11 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 11:
        alterado = documento
    elif len(documento) > 11:
        alterado = documento
    
    return alterado

In [47]:
nova = survey.apply(compara1, axis=1) #lê dataframe e chama a função

survey['cpf_inteiro'] = nova

In [48]:
survey.reset_index().head()

,index,respondent_id,collector_id,date_created,date_modified,ip_address,email_address,first_name,last_name,custom_1,...,url_instagram,url_youtube,financiamento_coletivo,passado_limpo,processos_primeira_instancia,detalhes_processos_survey,pacto,aderiu_totalmente_medidas,justificativa_adesao_medidas,cpf_inteiro
0,0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,Instagram,Youtube,Open-Ended Response,Response,Response,Open-Ended Response,Response,Response,Open-Ended Response,Open-Ended Response
1,1,1.026128e+10,215760080.0,2018-10-05 20:58:06,10/05/2018 08:59:11 PM,191.255.242.55,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000000000
2,2,1.026124e+10,215760080.0,2018-10-05 20:35:50,10/05/2018 08:37:46 PM,200.174.132.2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000000000
3,3,1.026120e+10,215760080.0,2018-10-05 20:14:09,10/05/2018 08:14:58 PM,179.176.175.61,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000000000
4,4,1.026119e+10,215760080.0,2018-10-05 20:11:03,10/05/2018 08:13:02 PM,179.159.23.23,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000000000


In [49]:
survey.to_excel('monkey/monkey_5out_2.xlsx',sheet_name='Sheet1')

# Importa Pacto

In [50]:
pacto = pd.read_excel("pacto/ADESÕES POLÍTICOS _ PACTO PELA DEMOCRACIA_5out_2.xlsx",sheet_name='ADESÕES SITE (nova)', converters={'CPF': lambda x: str(x)})

In [ ]:
# Elimina linhas sem CPF

In [51]:
pacto = pacto[pd.notnull(pacto['CPF'])]

In [ ]:
# Renomeia

In [52]:
pacto.rename(columns={"Status da adesão ": "adesao"}, inplace=True)
pacto.rename(columns={"Status vídeo": "video"}, inplace=True)

In [53]:
pacto.info()

<class 'pandas.core.frame.DataFrame'>
Index: 553 entries, helitec@uol.com.br to nan
Data columns (total 23 columns):
Nome                 546 non-null object
Sobrenome            539 non-null object
(End)                0 non-null float64
Telefone             415 non-null object
Estado               544 non-null object
Cidade               533 non-null object
Partido              526 non-null object
Movimento            237 non-null object
Cargo concorrente    543 non-null object
Facebook             482 non-null object
Instagram            222 non-null object
Twitter              23 non-null object
Youtube/Linkedin     16 non-null object
Site                 142 non-null object
Crowdfunding         19 non-null object
Mandato atual        500 non-null object
CPF                  553 non-null object
Nº urna              487 non-null object
Data de adesão       524 non-null object
adesao               553 non-null object
video                545 non-null object
Responsável          112 n

In [54]:
pacto.reset_index().head(5)

,index,Nome,Sobrenome,(End),Telefone,Estado,Cidade,Partido,Movimento,Cargo concorrente,...,Site,Crowdfunding,Mandato atual,CPF,Nº urna,Data de adesão,adesao,video,Responsável,Unnamed: 22
0,helitec@uol.com.br,Adelson,Bezerra,NaN,5519981196755,São Paulo,Campinas,NOVO,NaN,Deputado(a) Federal,...,www.adelson30.com.br,NaN,Não,79056547704,"3,026",8/20/18 7:21PM,SIM,Não enviado,NaN,NaN
1,ademarbueno10@gmail.com,ademar,bueno,NaN,11998356696,São Paulo,Santana de Parnaíba,REDE,Agora e RAPS,Deputado(a) Estadual,...,ademarbueno.com.br,NaN,Não,12774604850,NaN,8/16/18 3:08PM,SIM,Aprovado,NaN,NaN
2,adolfomariano@outlook.com,Adolfo,Mariano,NaN,16 991514058,São Paulo,Franca,PSOL,NaN,Deputado(a) Federal,...,NaN,NaN,Não,38927484894,5001,8/21/18 1:27PM,SIM,Não enviado,NaN,NaN
3,adolfur@yahoo.com.br,Adolfo Furtado Gonçalves,Gonçalves,NaN,(24)999648810,Rio de Janeiro,Volta Redonda,PODE,NaN,Deputado(a) Federal,...,NaN,NaN,Não,46987193720,"1,933",2018-12-09 06:06:00,Pendente,Não enviado,NaN,NaN
4,falecomigo@adrianaventura.com,Adriana,Ventura,NaN,1138629955,São Paulo,São Paulo,NOVO,NaN,Deputado(a) Federal,...,adrianaventura.com,NaN,Não,12519851813,"3,05",8/23/18 12:32AM,SIM,Não enviado,NaN,NaN


In [ ]:
# Tira símbolos do CPF

In [55]:
def limpeza1(linha):
    documento = str(linha["CPF"])
    documento = documento.strip()
    documento = documento.replace('.', '')
    documento = documento.replace('-', '')
    documento = documento.replace('/', '')
    documento = documento.replace(',', '')
    return str(documento)

In [56]:
pacto["cpf_limpo"] = pacto.apply(limpeza1, axis=1)

In [57]:
# Preenche zeros a esquerda de cpf
def compara2(linha):
    documento = str(linha['cpf_limpo']).strip()
    if documento == "":
        alterado = documento
    if len(documento) < 11:
        conta = 11 - (len(documento))
        zero = '0'
        zeros = zero * conta
        alterado = zeros + documento
    elif len(documento) == 11:
        alterado = documento
    elif len(documento) > 11:
        alterado = documento
    
    return alterado

In [58]:
nova = pacto.apply(compara2, axis=1) #lê dataframe e chama a função

pacto['cpf_inteiro'] = nova

In [59]:
pacto.reset_index().head()

,index,Nome,Sobrenome,(End),Telefone,Estado,Cidade,Partido,Movimento,Cargo concorrente,...,Mandato atual,CPF,Nº urna,Data de adesão,adesao,video,Responsável,Unnamed: 22,cpf_limpo,cpf_inteiro
0,helitec@uol.com.br,Adelson,Bezerra,NaN,5519981196755,São Paulo,Campinas,NOVO,NaN,Deputado(a) Federal,...,Não,79056547704,"3,026",8/20/18 7:21PM,SIM,Não enviado,NaN,NaN,79056547704,79056547704
1,ademarbueno10@gmail.com,ademar,bueno,NaN,11998356696,São Paulo,Santana de Parnaíba,REDE,Agora e RAPS,Deputado(a) Estadual,...,Não,12774604850,NaN,8/16/18 3:08PM,SIM,Aprovado,NaN,NaN,12774604850,12774604850
2,adolfomariano@outlook.com,Adolfo,Mariano,NaN,16 991514058,São Paulo,Franca,PSOL,NaN,Deputado(a) Federal,...,Não,38927484894,5001,8/21/18 1:27PM,SIM,Não enviado,NaN,NaN,38927484894,38927484894
3,adolfur@yahoo.com.br,Adolfo Furtado Gonçalves,Gonçalves,NaN,(24)999648810,Rio de Janeiro,Volta Redonda,PODE,NaN,Deputado(a) Federal,...,Não,46987193720,"1,933",2018-12-09 06:06:00,Pendente,Não enviado,NaN,NaN,46987193720,46987193720
4,falecomigo@adrianaventura.com,Adriana,Ventura,NaN,1138629955,São Paulo,São Paulo,NOVO,NaN,Deputado(a) Federal,...,Não,12519851813,"3,05",8/23/18 12:32AM,SIM,Não enviado,NaN,NaN,12519851813,12519851813


In [ ]:
# Cria nova coluna a partir de condições

In [60]:
def compara3(linha):
    aceite = str(linha['adesao']).strip()
    #print(aceite)
    if aceite == '' or (isinstance(aceite, float) and math.isnan(aceite)):
        alterado = 'SEM RESPOSTA'
    elif aceite == 'VAZIO':
        alterado = 'SEM RESPOSTA'
    elif aceite == "SIM":
        alterado = 'SIM'
    elif aceite == "email voltou":
        alterado = 'SEM RESPOSTA'
    elif aceite == "Pendente":
        alterado = 'SEM RESPOSTA'
    elif aceite == "Não":
        alterado = 'NÃO'
    
    return alterado

In [61]:
nova = pacto.apply(compara3, axis=1) #lê dataframe e chama a função

pacto['aceite_ajustado'] = nova

In [62]:
pacto.reset_index().head()

,index,Nome,Sobrenome,(End),Telefone,Estado,Cidade,Partido,Movimento,Cargo concorrente,...,CPF,Nº urna,Data de adesão,adesao,video,Responsável,Unnamed: 22,cpf_limpo,cpf_inteiro,aceite_ajustado
0,helitec@uol.com.br,Adelson,Bezerra,NaN,5519981196755,São Paulo,Campinas,NOVO,NaN,Deputado(a) Federal,...,79056547704,"3,026",8/20/18 7:21PM,SIM,Não enviado,NaN,NaN,79056547704,79056547704,SIM
1,ademarbueno10@gmail.com,ademar,bueno,NaN,11998356696,São Paulo,Santana de Parnaíba,REDE,Agora e RAPS,Deputado(a) Estadual,...,12774604850,NaN,8/16/18 3:08PM,SIM,Aprovado,NaN,NaN,12774604850,12774604850,SIM
2,adolfomariano@outlook.com,Adolfo,Mariano,NaN,16 991514058,São Paulo,Franca,PSOL,NaN,Deputado(a) Federal,...,38927484894,5001,8/21/18 1:27PM,SIM,Não enviado,NaN,NaN,38927484894,38927484894,SIM
3,adolfur@yahoo.com.br,Adolfo Furtado Gonçalves,Gonçalves,NaN,(24)999648810,Rio de Janeiro,Volta Redonda,PODE,NaN,Deputado(a) Federal,...,46987193720,"1,933",2018-12-09 06:06:00,Pendente,Não enviado,NaN,NaN,46987193720,46987193720,SEM RESPOSTA
4,falecomigo@adrianaventura.com,Adriana,Ventura,NaN,1138629955,São Paulo,São Paulo,NOVO,NaN,Deputado(a) Federal,...,12519851813,"3,05",8/23/18 12:32AM,SIM,Não enviado,NaN,NaN,12519851813,12519851813,SIM


In [63]:
pacto.to_excel('pacto/pacto_5out_2.xlsx',sheet_name='Sheet1')